In [1]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

716789
705921


In [3]:
df = df.filter(pl.col("stran") >= 30)

In [4]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

364420

In [5]:
vek = pl.read_parquet(os.path.join("data","narozeni-umrti-gender.parquet"))
df = df.join(vek, on="100_7", how="left").with_columns((pl.col("rok") - pl.col("narozeni")).alias("vek"))

In [6]:
df_700 = pl.read_parquet(os.path.join("data","podil_lidi_s_pauzou.parquet")).with_columns(pl.col("rok").map_elements(lambda x: x.year)).filter(pl.col("rok").is_between(1910,1990))

In [7]:
alt.Chart(alt_friendly(df_700)).mark_line().encode(alt.X("rok:T"), alt.Y("podil_cechu_ill_10"))

alt.Chart(...)

In [8]:
alt.Chart(alt_friendly(df_700)).mark_line().encode(alt.X("rok:T"), alt.Y("podil_cechu_trl_10"))

alt.Chart(...)

## Autorstvo s pauzou a končící autorstvo

In [10]:
df_100 = pl.read_parquet(os.path.join("data","podil_autorstva_s_pauzou.parquet")).with_columns(pl.col("rok").map_elements(lambda x: x.year)).filter(pl.col("rok").is_between(1900,2000)).with_columns(pl.col("podil_cechu_15").map_elements(lambda x: round(x, 3)))
df_koncici = pl.read_parquet(os.path.join('data','podily_konciciho_autorstva_cs.parquet'))

### Záhada roku 1905

In [12]:
df = df.with_columns(pl.col("100_a").map_elements(hezke_jmeno).alias("jmeno"))

In [13]:
rok04 = df.filter(pl.col("rok") == 1904).select(pl.col("100_7")).to_series().to_list()
roky06az18 = df.filter(pl.col("rok").is_between(1906,1918)).select(pl.col("100_7")).to_series().to_list()
roky18plus = df.filter(pl.col("rok") > 1900).select(pl.col("100_7")).to_series().to_list()
odmlceni_05 = [x for x in rok04 if x in roky18plus and x not in roky06az18]

In [14]:
df.filter(pl.col('100_7').is_in(odmlceni_05)).group_by(['jmeno','100_7']).len().sort(by='len',descending=True).head(20)

jmeno,100_7,len
str,str,u32
"""Ladislav Klíma""","""jk01060531""",75
"""Antonín Gindely""","""jk01032434""",50
"""Václav Vladivoj Tomek""","""jk01132708""",40
"""František Cajthaml-Liberté""","""jk01020537""",25
"""František Vlasák""","""jk01150047""",24
"""Adolf Eckert""","""jk01030294""",22
"""Martin von Cochem""","""jo2002109710""",22
"""František Cyril Kampelík""","""jk01052820""",21
"""Quintus Flaccus Horatius""","""jn20000603003""",19


In [15]:
', '.join(df.filter(pl.col('100_7').is_in(odmlceni_05)).group_by(['jmeno','100_7']).len().sort(by='len',descending=True).select(pl.col('jmeno')).to_series().to_list()[0:30])

'Ladislav Klíma, Antonín Gindely, Václav Vladivoj Tomek, František Cajthaml-Liberté, František Vlasák, Martin von Cochem, Adolf Eckert, František Cyril Kampelík, Quintus Flaccus Horatius, Hendrik Conscience, Antonín Konstantin Viták, Jan Karník, Jan Šimsa, Matyáš Blažek, František Ladislav Menhard, Eugen Kumičić, Luboš Jeřábek, František Karel Bakule, Leopold M. Zeithammer, Václav Vondrák, František Buben, Josef Šmaha, Martin Kuchynka, Robert Saudek, Václav Franěk, Josef Barák, Rudolf Knaus, E. V. Řičák, Jan Sommer, Eduard Georgievič Valečka'

In [16]:
graf_pauzy = alt.Chart(
    alt_friendly(df_100.filter(pl.col('rok').is_between(1930,1995))), width=kredity['sirka'], height=kredity['vyska_nizkych'] * 1.5,
    title=alt.Title("Podíl českých autorů, kteří se odmlčeli na 10 a více let")
).mark_bar(color='#E09DA3', width=3,
).encode(alt.X(
    "rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
), alt.Y(
    "podil_cechu_10:Q", title=None, 
    axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label * 100 + ' %'", orient='right', tickCount=3),
    scale=alt.Scale(domain=[0,0.095])
))
                    
rule1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

rule2 = alt.Chart(alt.Data(values=[{'rok': '1968-08-21'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

# Add text annotation
text1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25', 'y': 0.13}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    dy=5,# Slight horizontal offset from the line
    dx=-4,
    fontSize = 8,
    text=['komunistický','převrat'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)


# Add text annotation
text2 = alt.Chart(alt.Data(values=[{'rok': '1965-08-21', 'y': 1}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    dy=5,# Slight horizontal offset from the line
    dx=12,
    fontSize=8,
    text=['srpnová','okupace'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

graf_pauzy_graf_komb = graf_pauzy + rule1 + text1 + rule2 + text2
graf_pauzy_graf_komb

alt.LayerChart(...)

In [17]:
graf_konce = alt.Chart(
    alt_friendly(df_koncici.filter(pl.col("rok").is_between(1900,2000))
                ), width=kredity['sirka'], height=kredity['vyska_nizkych'] * 1.5,
                 title=alt.Title(["Podíl českých autorů, kteří vydali svoji poslední knihu"])
).mark_bar(color='#E09DA3',width=3
).encode(
    alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Y("podil:Q", title=None, scale=alt.Scale(domain=[0,0.25]), axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', tickCount=3, labelExpr="datum.label * 100 + ' %'", orient='right')))
graf_konce

rule1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

rule2 = alt.Chart(alt.Data(values=[{'rok': '1989-11-17'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

# Add text annotation
text1 = alt.Chart(alt.Data(values=[{'rok': '1952-02-25', 'y': 0.2}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='left',
    baseline='middle',
    dy=2,# Slight horizontal offset from the line
    dx=-9,
    fontSize = 8,
    text=['komunistický','převrat'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)


# Add text annotation
text2 = alt.Chart(alt.Data(values=[{'rok': '1987-11-17', 'y': 0.2}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    fontSize=8,
    dy=2,# Slight horizontal offset from the line
    dx=-2,
    text=['sametová','revoluce'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

graf_konce_komb = graf_konce + text1 + text2

graf_konce_komb

alt.LayerChart(...)

In [18]:
pauzy_konce_final = alt.vconcat(
    graf_konce_komb, graf_pauzy_graf_komb
).configure_view(
    stroke='transparent'
).resolve_scale(
    y='shared'
).configure_axisX(
    grid=False, domain=False
)

pauzy_konce_final

alt.VConcatChart(...)

In [19]:
konce_final = graf_konce_komb.configure_view(
    stroke='transparent'
).resolve_scale(
    y='shared'
).configure_axisX(
    grid=False, domain=False
)

konce_final

alt.LayerChart(...)

In [20]:
pauzy_final = graf_pauzy_graf_komb.configure_view(
    stroke='transparent'
).resolve_scale(
    y='shared'
).configure_axisX(
    grid=False, domain=False
)

pauzy_final

alt.LayerChart(...)

In [21]:
me_to_neurazi(konce_final, soubor="04_konce", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_konce.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_konce.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [22]:
me_to_neurazi(pauzy_final, soubor="04_pauzy", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_pauzy.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_pauzy.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [23]:
# me_to_neurazi(pauzy_konce_final, soubor="04_konce_pauzy", kredity=kredity['default'])

## Kdo zmizel

In [25]:
pred_unorem = set(df.filter(pl.col("rok") <= 1948).select(pl.col("100_7")).to_series().to_list())
po_revoluci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("100_7")).to_series().to_list())
v_sedesatkach = set(df.filter(pl.col("rok").is_between(1960,1969)).select(pl.col("100_7")).to_series().to_list())
behem_socialismu = set(df.filter(pl.col("rok").is_between(1949,1987)).select(pl.col("100_7")).to_series().to_list())
behem_padesatek = set(df.filter(pl.col("rok").is_between(1949,1959)).select(pl.col("100_7")).to_series().to_list())
pred_normalizaci = set(df.filter(pl.col("rok").is_between(1966,1970)).select(pl.col("100_7")).to_series().to_list())
po_normalizaci = set(df.filter(pl.col("rok").is_between(1990,1994)).select(pl.col("100_7")).to_series().to_list())
behem_normalizace = set(df.filter(pl.col("rok").is_between(1972,1988)).select(pl.col("100_7")).to_series().to_list())

znormalizovani = pred_normalizaci.intersection(po_normalizaci).difference(behem_normalizace)
zunorovani = pred_unorem.intersection(po_revoluci).difference(behem_socialismu)

In [26]:
len(znormalizovani)

508

In [27]:
len(df.filter(pl.col("100_7").is_in(set(cesi).intersection(znormalizovani))).unique(subset=['100_7']))

397

In [145]:
kolik = 23
znormalizovani_top_domaci = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
znormalizovani_top_domaci_razeni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
znormalizovani_top_zahranicni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
znormalizovani_top_zahranicni_razeni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
znormalizovani_top_domaci_razeni

['Josef Škvorecký',
 'Ivan Sviták',
 'Ivan Klíma',
 'Václav Havel',
 'Karel Kaplan',
 'Anna Sedlmayerová',
 'Vladimír Škutina',
 'Jan Vladislav',
 'Karel Šiktanc',
 'Jan Procházka',
 'Václav Černý',
 'Jan Patočka',
 'Václav Renč',
 'Ivan Diviš',
 'Milan Kundera',
 'Eva Kantůrková',
 'Adolf Melezinek',
 'Arnošt Lustig',
 'Karel Pecka',
 'František Kautman',
 'Ludvík Vaculík',
 'Zdeněk Kalista',
 'Václav Klaus']

In [29]:
# kolik = 20
# znormalizovani_top_domaci = df.filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
# znormalizovani_top_domaci_razeni = df.filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
# znormalizovani_top_zahranicni = df.filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
# znormalizovani_top_zahranicni_razeni = df.filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()

Načteme data vygenerovaná sešitem 174_zahranicni_vydani.ipynb.

In [51]:
horni_hranice = 2015

In [149]:
dografu2 = pl.read_parquet(os.path.join('data','zahranicni_knihy_10_zakazanych_autoru.parquet')).select(pl.col(['jmeno','titul','rok'])).with_columns(pl.lit('zahranici').alias('kde')).filter(pl.col("rok").is_between(1930,horni_hranice)).filter(pl.col("jmeno").is_in(znormalizovani_top_domaci_razeni))

In [151]:
dografu1 = df.filter(pl.col("rok").is_between(1930,horni_hranice)).filter(pl.col("100_7").is_in(znormalizovani_top_domaci)).select(pl.col(['jmeno','245_a','rok'])).rename({'245_a':'titul'}).with_columns(pl.lit('domaci').alias('kde'))

In [153]:
dografu = pl.concat([dografu1, dografu2]).filter(pl.col("jmeno").is_in(znormalizovani_top_domaci_razeni))

dografu = dografu.join(
    df.filter(
        pl.col("rok").is_between(1930,horni_hranice)
    ).filter(
        pl.col("100_7").is_in(znormalizovani_top_domaci)
    ).group_by('jmeno').agg(pl.col('umrti').max()), on='jmeno',how='left'
)

dografu = dografu.with_columns(
    pl.when(pl.col('umrti') > horni_hranice).then(None).otherwise(pl.col('umrti')).alias("umrti")
).with_columns(
    pl.col('umrti').map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime)
)

In [155]:
dografu

jmeno,titul,rok,kde,umrti
str,str,i64,str,datetime[μs]
"""Jan Patočka""","""Fenomenologické spisy""",2009,"""domaci""",1977-01-01 00:00:00
"""Arnošt Lustig""","""Colette, dívka z Antverp""",2005,"""domaci""",2011-01-01 00:00:00
"""Eva Kantůrková""","""Černá hvězda""",1992,"""domaci""",null
"""Karel Kaplan""","""Příprava Ústavy ČSR v letech 1…",1993,"""domaci""",null
"""Jan Patočka""","""Platón a Evropa""",2007,"""domaci""",1977-01-01 00:00:00
"""Ivan Klíma""","""Mistr""",1967,"""domaci""",null
"""Václav Renč""","""Ve spleti větví čtu tvá ramena""",2011,"""domaci""",1973-01-01 00:00:00
"""Václav Renč""","""Perníková chaloupka""",1944,"""domaci""",1973-01-01 00:00:00
"""Milan Kundera""","""Směšné lásky""",2007,"""domaci""",null


In [163]:
# First create the base chart parameters
base = alt.Chart(alt_friendly(dografu.filter(pl.col("rok").is_between(1945,horni_hranice))), 
    title=alt.TitleParams(
        ["Normalizační průrva českou literaturou"], 
        subtitle=["Čeští autoři a autorky žijící v době srpnové okupace, kterým toho vyšlo nejvíc",
                  "v druhé půlce 60. let, první půlce 90. let a zároveň nic mezi lety 1973 a 1987.",
                  "Červené tečky jsou knihy vydané na českém území, modré vydané v zahraničí,",
                  "černé čárky jsou roky úmrtí a barevný pruh doba mezi okupací a revolucí."]
    ), width=kredity['sirka'] / 1.25)

# Create the yellow dots layer (foreign publications)
foreign = base.mark_circle(opacity=1).encode(
    x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white')),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    size=alt.value(6),
    color=alt.value('#81A9D5')
).transform_filter(
    alt.datum.kde == 'zahranici'
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

# Create the domestic publications layer
domestic = base.mark_circle(opacity=1).encode(
    x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white')),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    size=alt.value(9),
    color=alt.Color('domaci:N', 
                   scale=alt.Scale(range=['#D6534B']), 
                   sort=znormalizovani_top_domaci_razeni,
                   legend=None)
).transform_filter(
    alt.datum.kde == 'domaci'
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

kdy_umreli = base.mark_tick(
    color='black',  # optional: you can specify color
    thickness=1.75,
    height=9
).encode(
    x=alt.X('umrti:T', title=None),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white'))
)

okupace_cara = alt.Chart(alt.Data(values=[{'rok': '1968-08-21'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

revoluce_cara = alt.Chart(alt.Data(values=[{'rok': '1989-11-17'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

okupace_text = alt.Chart(alt.Data(values=[{'rok': '1968-08-21', 'y': 0.2}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    fontSize=8,
    dy=2,# Slight horizontal offset from the line
    dx=-2,
    text=['sametová','revoluce'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

hranice_normalizace = pd.DataFrame([{"start": "1968-08-21", "end": "1989-11-17", "event": "normalizace"}])

rect = alt.Chart(hranice_normalizace).mark_rect(color='#ecf2f8').encode(
    x="start:T",
    x2="end:T"
)

# Layer the charts with foreign (yellow) on bottom and domestic on top
norm_dom = alt.layer(rect, kdy_umreli, foreign, domestic).configure_view(stroke='transparent').resolve_scale(color='independent',x="shared").configure_axisX(grid=False) # .configure_axisY(grid=True)

norm_dom

alt.LayerChart(...)

In [165]:
me_to_neurazi(norm_dom, kredity=kredity['default'], soubor="04_normalizace")

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/04_normalizace.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/04_normalizace.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [123]:
znormalizovani_top_zahranicni_razeni

['Erich von Däniken',
 'Ladislav Mňačko',
 'Peter Ferdinand Drucker',
 'Jean-Paul Sartre',
 'Simone de Beauvoir',
 'Erich Fromm',
 'Max Frisch',
 'Rochdi Goulli',
 'Jean Genet',
 'Ladislav Grosman',
 'Richmal Crompton',
 'Eugène Ionesco',
 'Clifford D. Simak',
 'Peter Weiss',
 'Poul Anderson',
 'John Knittel',
 'Günter Grass',
 'Nathalie Sarraute',
 'Marguerite Duras',
 'Karl Jaspers',
 'André Pieyre de Mandiargues',
 'Wolfgang Ott',
 'Jan Blahoslav Čapek',
 'Aischylos',
 'Cornelius Ryan']

## Stopky

In [ ]:
nechceme = ['jn19990210621']
df = df.filter(~pl.col('100_7').is_in(nechceme))

In [ ]:
def nevysli_po(rok, obdobi, nasobek):
    vysli_pred = df.filter(pl.col("rok").is_between(rok-obdobi,rok)).group_by('100_7').len().filter(pl.col('len') > 3).select(pl.col("100_7")).to_series().to_list()
    vysli_po = set(df.filter(pl.col("rok") > rok).select(pl.col("100_7")).to_series().to_list())
    ukonceni = [v for v in vysli_pred if v not in vysli_po]
    print(f"Filtru vyhovuje {len(ukonceni)} lidí.")
    return df.filter(pl.col("100_7").is_in(ukonceni)).filter(pl.col("rok").is_between(rok-(obdobi * nasobek),rok)).group_by(["jmeno","100_7"]).len().sort(by="len",descending=True).head(100)

In [ ]:
sedesatosm = nevysli_po(1968, 2, 40).head(3)
sedesatosm

In [ ]:
tricetosm = nevysli_po(1938, 2, 40).head(3)
tricetosm

In [ ]:
ctyricetosm = nevysli_po(1948, 4, 40).head(3)
ctyricetosm

In [ ]:
osmnact = nevysli_po(1918, 4, 40).head(3)
osmnact

In [ ]:
ctyricetosm = nevysli_po(1948, 4, 40).head(3)
ctyricetosm

In [ ]:
nevysli_po(1968, 4, 40).head(3)

In [ ]:
tricetdevet = nevysli_po(1939, 4, 40).head(3)
tricetdevet

In [ ]:
osmdesatdevet = nevysli_po(1989, 4, 40).head(3)
osmdesatdevet

In [ ]:
df = df.with_columns(pl.col("jmeno").replace({"Ladislav Riedl Německobrodský": "L. R. Německobrodský"}))
df = df.with_columns(pl.col("jmeno").replace({"Il‘ja Grigor‘jevič Èrenburg": "Ilja Erenburg"}))
df = df.with_columns(pl.col("jmeno").replace({"Vasilij Ivanovič Nemirovič-Dančenko": "Nemirovič-Dančenko"}))
df = df.with_columns(pl.col("jmeno").replace({"Albert Vojtěch Velflík": "A. V. Velfík"}))

In [ ]:
#fertig = pl.concat([osmnact, ctyricetosm, osmdesatdevet])

fertig_18_kdo = osmnact.select(pl.col('100_7')).to_series().to_list()
fertig_18_razeni = osmnact.select(pl.col('jmeno')).to_series().to_list()
fertig_18_do_grafu = df.filter(pl.col('100_7').is_in(fertig_18_kdo))

fertig_39_kdo = tricetdevet.select(pl.col('100_7')).to_series().to_list()
fertig_39_razeni = tricetdevet.select(pl.col('jmeno')).to_series().to_list()
fertig_39_do_grafu = df.filter(pl.col('100_7').is_in(fertig_39_kdo))

fertig_48_kdo = ctyricetosm.select(pl.col('100_7')).to_series().to_list()
fertig_48_razeni = ctyricetosm.select(pl.col('jmeno')).to_series().to_list()
fertig_48_do_grafu = df.filter(pl.col('100_7').is_in(fertig_48_kdo))

fertig_89_kdo = osmdesatdevet.select(pl.col('100_7')).to_series().to_list()
fertig_89_razeni = osmdesatdevet.select(pl.col('jmeno')).to_series().to_list()
fertig_89_do_grafu = df.filter(pl.col('100_7').is_in(fertig_89_kdo))

In [ ]:
fertig_48_razeni

In [ ]:
fertig_18_graf = alt.Chart(alt_friendly(fertig_18_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(
    ["Nejvydávanější čeští autoři, kteří přestali vycházet","po vyhlášení Československa…"], dx=-22,
)).mark_circle(size=8) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_18_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_18_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_39_graf = alt.Chart(alt_friendly(fertig_39_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(["…po nacistické okupaci…"], 
)).mark_circle(size=8) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_39_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_39_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_48_graf = alt.Chart(alt_friendly(fertig_48_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(["…po únorovém převratu…"], 
)).mark_circle(size=8) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_48_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_48_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_89_graf = alt.Chart(alt_friendly(fertig_89_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(
    ["…a po revoluci"], dx=-11
)).mark_circle(size=8) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_89_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=fertig_89_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

zmenseni = 1.25

seznamy_koncicich = alt.vconcat(
    fertig_18_graf.properties(
    width=kredity['sirka'] / zmenseni), 
    fertig_39_graf.properties(
    width=kredity['sirka'] / zmenseni), 
    fertig_48_graf.properties(
    width=kredity['sirka'] / zmenseni), 
    fertig_89_graf.properties(
    width=kredity['sirka'] / zmenseni)
).resolve_axis(x="shared").configure_view(stroke='transparent')

seznamy_koncicich

In [ ]:
me_to_neurazi(seznamy_koncicich, soubor="04_seznamy_koncicich", kredity=kredity['default'])

In [ ]:
df.filter(pl.col("jmeno") == "Jarmila Šteflíčková")

## Lze najít rozdíl napsáno-vydáno?

In [ ]:
df.filter(pl.col("245_a").str.contains("Zbabělci"))